# Import necessary libraries

In [8]:
import snowflake.connector
import configparser
import os
from dotenv import load_dotenv
import pandas as pd

In [2]:
config = configparser.ConfigParser()
config.read('config.env')

['config.env']

In [3]:
load_dotenv()

False

# Get Connection Details   

In [4]:
conn = snowflake.connector.connect(
    user=config['snowflake']['user'],     
    password=config['snowflake']['password'], 
    account=config['snowflake']['account'],    
    warehouse=config['snowflake']['warehouse'],  
    database=config['snowflake']['database'],    
    schema=config['snowflake']['schema'],        
    client_session_keep_alive=True
)

# Establish snowflake connection

In [5]:
print("Connecting to Snowflake...")
cursor = conn.cursor()

Connecting to Snowflake...


# Queries to answer some business questions

In [15]:
# Query 1: Most popular transport types
most_popular_types_query = """
SELECT journey_type,
    SUM(journeys_millions)/1000000 as total_journeys_millions
FROM KBELE_LONDON.PUBLIC.JOURNEYS
GROUP BY journey_type
ORDER BY total_journeys_millions DESC;
"""

# Query 2: Emirates Airline popularity by month and year
emirates_query = """
SELECT month, 
	year, 
	ROUND(journeys_millions,2) AS rounded_journeys_millions
FROM KBELE_LONDON.PUBLIC.JOURNEYS
WHERE journey_type = 'Emirates Airline' AND journeys_millions IS NOT NULL
ORDER BY rounded_journeys_millions DESC
LIMIT 5;
"""

# Query 3: Least popular years for Underground & DLR
underground_query = """
SELECT year,
	journey_type,
	SUM(journeys_millions) as total_journeys_millions
FROM KBELE_LONDON.PUBLIC.JOURNEYS
WHERE journey_type LIKE '%Underground%'
GROUP BY year, journey_type
ORDER BY total_journeys_millions
LIMIT 5;
"""

In [9]:

print("\nMost Popular Transport Types:")
cursor.execute(most_popular_types_query)
results1 = cursor.fetchall()
df_popular_types = pd.DataFrame(results1, columns=['journey_type', 'total_journeys_millions'])
print(df_popular_types)


Most Popular Transport Types:
        journey_type total_journeys_millions
0                Bus          0.024905193947
1  Underground & DLR          0.015020466544
2         Overground          0.001666845666
3           TfL Rail          0.000411313421
4               Tram          0.000314689875
5   Emirates Airline          0.000014583718


In [14]:
print("\nTop 5 Months for Emirates Airline:")
cursor.execute(emirates_query)
results2 = cursor.fetchall()
df_top = pd.DataFrame(results2, columns=['Month', 'Year', 'rounded_journeys_millions'])
print(df_top)



Top 5 Months for Emirates Airline:
   Month  Year rounded_journeys_millions
0      5  2012                      0.53
1      6  2012                      0.38
2      4  2012                      0.24
3      5  2013                      0.19
4      5  2015                      0.19


In [16]:
print("\nLeast Popular Years for Underground & DLR:")
cursor.execute(underground_query)
results3 = cursor.fetchall()
df_least_pop = pd.DataFrame(results3, columns=['Year', 'journey_type', 'total_journeys_millions'])
print(df_least_pop)


Least Popular Years for Underground & DLR:
   Year       journey_type total_journeys_millions
0  2020  Underground & DLR          310.1793163140
1  2021  Underground & DLR          748.4525442000
2  2022  Underground & DLR         1064.8590086000
3  2010  Underground & DLR         1096.1455883800
4  2011  Underground & DLR         1156.6476544800


# Execute first query

In [18]:
finally:
cursor.close()
conn.close()

SyntaxError: invalid syntax (686037153.py, line 1)